In [1]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [32]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader, UnstructuredEPubLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [17]:
import os

directory = 'data'
docs = []

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(directory, filename))
        doc = loader.load_and_split()
        docs.extend(doc)

In [19]:
print(docs[0].page_content[:500])

I BATSFORD CHESS I 
IRVING CHERNEV


In [20]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

2166

In [21]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [33]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)
db.save_local("vector_db")
print("Local vectorized db has been successfully saved.")

Local vectorized db has been successfully saved.


In [22]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What is the chess?")

len(retrieved_docs)

6

In [23]:
print(retrieved_docs[0].page_content.replace("\t", " "))

thought that the simple game of Chaturanga would evolve into one of the most
popular games in the world today, chess!
Chessboard And Chess Pieces
Before you start playing chess, it is imperative that you understand the board
and what each piece on the board represents. All the pieces need to be arranged in a
correct fashion and only then can the games begin. Let us look at the board and
pieces in detail.
The Chessboard


In [24]:
prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [25]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("How do I take control of the centre in chess?"):
    print(chunk, end="", flush=True)

To take control of the center in chess, you must place your pawns strategically to dominate important squares and attack others. Develop your high-value pieces quickly and spread them out on the board evenly. Understanding the values of each piece and their placement will help you conquer the center effectively.

In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "How do I take control of the centre in chess?"})
print(response["answer"])

To take control of the center in chess, it is essential to have a command over the central squares of the board. Placing pawns and pieces strategically in the center allows you to dominate the board. You can control the center by developing your pieces quickly, memorizing opening strategies, and understanding the value of each piece. Additionally, knowing when to open or block the center based on the game situation is crucial. By controlling the center, you can effectively plan your moves and gain an advantage over your opponent.
